In [ ]:
"""Cartpole DQN models."""
import tensorflow as tf
from keras.layers import Embedding, Dense, Layer, LSTM, Input, RNN
from tensorflow.keras.optimizers import Adam
from keras.models import Model
import keras.backend as K
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class LSTMCell(Layer):

    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(LSTMCell, self).__init__(**kwargs)
        implementation = kwargs.pop('implementation', 1)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(input_dim, self.units * 4), name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units * 4),name='recurrent_kernel')
        self.bias = self.add_weight(shape=(self.units * 4), name='bias')
        self.built = True
    
    def _compute_carry_and_output(self, x, h_tm1, c_tm1):
      #"""Computes carry and output using split kernels."""
      x = x_i, x_f, x_c, x_o 
      h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o = h_tm1
      i = self.recurrent_activation(
        x_i + K.dot(h_tm1_i, self.recurrent_kernel[:, :self.units]))
      f = self.recurrent_activation(x_f + backend.dot(
        h_tm1_f, self.recurrent_kernel[:, self.units:self.units * 2]))
      c = f * c_tm1 + i * self.activation(x_c + backend.dot(
        h_tm1_c, self.recurrent_kernel[:, self.units * 2:self.units * 3]))
      o = self.recurrent_activation(
          x_o + K.dot(h_tm1_o, self.recurrent_kernel[:, self.units * 3:]))
      return c, o

    
    def call(self, inputs, states, training=None):
      h_tm1 = states[0]  # previous memory state
      c_tm1 = states[1]  # previous carry state

      if self.implementation == 1:
        inputs_i = inputs
        inputs_f = inputs
        inputs_c = inputs
        inputs_o = inputs

        k_i, k_f, k_c, k_o = tf.split(self.kernel, num_or_size_splits=4, axis=1)
        x_i = K.dot(inputs_i, k_i)
        x_f = K.dot(inputs_f, k_f)
        x_c = K.dot(inputs_c, k_c)
        x_o = K.dot(inputs_o, k_o)

      if self.use_bias:
        b_i, b_f, b_c, b_o = tf.split(
            self.bias, num_or_size_splits=4, axis=0)
        x_i = K.bias_add(x_i, b_i)
        x_f = K.bias_add(x_f, b_f)
        x_c = K.bias_add(x_c, b_c)
        x_o = K.bias_add(x_o, b_o)

      x = (x_i, x_f, x_c, x_o)
      h_tm1 = (h_tm1_i, h_tm1_f, h_tm1_c, h_tm1_o)
      c, o = self._compute_carry_and_output(x, h_tm1, c_tm1)

      h = o * self.activation(c)
      return h, [h, c]

In [ ]:
#from rnn import RNN
batch_size = 1
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 2

units = 5
output_size = 1  # labels are from 0 to 9

# Build the LSTM model
def build_model():           #allow_cudnn_kernel=True
  """Build a stateful simple LSTM model."""
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
            # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
  lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(None, input_dim)
        )
  model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.Dense(output_size),
        ]
    )
  return model


In [15]:
batch_size = 1
# Each MNIST image batch is a tensor of shape (batch_size, 28, 28).
# Each input sequence will be of size (28, 28) (height is treated like time).
input_dim = 1

units = 5
output_size = 1  # labels are from 0 to 9

# Build the LSTM model
    # CuDNN is only available at the layer level, and not at the cell level.
    # This means `LSTM(units)` will use the CuDNN kernel,
    # while RNN(LSTMCell(units)) will run on non-CuDNN kernel.
            # Wrapping a LSTMCell in a RNN layer will not use CuDNN.
lstm_layer = keras.layers.RNN(
            keras.layers.LSTMCell(units), input_shape=(10, input_dim)
        )
model = keras.models.Sequential(
        [
            lstm_layer,
            keras.layers.Dense(output_size),
        ]
    )
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rnn_1 (RNN)                 (None, 5)                 140       
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
%cd /content/drive/My Drive/ECE570/code/

/content/drive/My Drive/ECE570/code


In [25]:
!pwd

/content/drive/My Drive/ECE570/code


In [18]:
from google.colab import files
files.upload()

{}

In [26]:
file_name = "/content/drive/My Drive/ECE570/code/sample1.txt"
raw_text = open(file_name, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length,1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length))
# normalize
X = X / float(1.5*n_vocab)
print(X.shape)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(y.shape)

Total Characters:  607
Total Vocab:  27
Total Patterns:  597
(597, 10)
(597, 27)


In [27]:
from sklearn.model_selection import train_test_split
# define the LSTM model
#model = Sequential()
#model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#model.add(Dropout(0.2))
#model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights_pendulum_rnn-{epoch:02d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y[:,1], test_size=0.2, random_state=42,stratify=y[:,1])
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=5, batch_size=10, callbacks=callbacks_list)

Epoch 1/5
44/48 [==========================>...] - ETA: 0s - loss: 1.6527e-08
Epoch 00001: loss improved from inf to 0.00000, saving model to pendulum_new_weights_rnn-01-0.0000.hdf5
48/48 [==============================] - 2s 11ms/step - loss: 1.6994e-08 - val_loss: 1.6888e-08
Epoch 2/5
46/48 [===========================>..] - ETA: 0s - loss: 1.7104e-08
Epoch 00002: loss did not improve from 0.00000
48/48 [==============================] - 0s 6ms/step - loss: 1.6994e-08 - val_loss: 1.6888e-08
Epoch 3/5
47/48 [============================>.] - ETA: 0s - loss: 1.6994e-08
Epoch 00003: loss did not improve from 0.00000
48/48 [==============================] - 0s 5ms/step - loss: 1.6994e-08 - val_loss: 1.6888e-08
Epoch 4/5
46/48 [===========================>..] - ETA: 0s - loss: 1.6326e-08
Epoch 00004: loss did not improve from 0.00000
48/48 [==============================] - 0s 5ms/step - loss: 1.6994e-08 - val_loss: 1.6888e-08
Epoch 5/5
47/48 [============================>.] - ETA: 0s - l